In [1]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
import statistics
import seaborn as sns; sns.set(style="whitegrid")
import math

%matplotlib inline

In [4]:
x = [1.3,1.6,1.8,2.1,2.3,2.7]
y = [4.1,4.7,5.8,5.9,6.5,7.7]

x=[10.55]
y=[14.26]
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()

In [5]:
model.params

array([1.35165877])

In [15]:
import test as t

In [16]:
t.adicao(1, 2)

AttributeError: module 'test' has no attribute 'adicao'

In [12]:
class Teste:
    def soma(self, v1, v2):
        return v1+v2

In [13]:
t = Teste()

In [14]:
t.soma(2, 3)

5